# Build Region of Interest with GeoPandas

We will build a new shapefile containing the Region of Interest (ROI) from a set of coordinates.

### Choose the CRS of the new shapefile - crs_dst

It is important to set the EPSG code matching with the EPSG code of your satellite images!

For instance, if your ROI is located in Belgium,
- the CRS is WGS 84 / UTM zone 31N
- the EPSG code is 32631

### Choose the CRS of the coordinates - crs_src

- Degree - WGS84 - EPSG:4326
- Meters - WGS 84 / UTM zone 31N - EPSG:32631 (RECOMMANDED)

### Coordinates bounding box

Extent in QGIS --> ulx, uly : lrx, lry

- ulx = Upper Left x
- uly = Upper Left y
- lrx = Lower Right x
- lry = Lower Right y


<img src="figures/ROI_box.png" width="500">

In [11]:
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from pathlib import Path
import IPython.display
from IPython.display import display
from sentinelhub import BBox

print(f'Pandas    : {pd.__version__}')
print(f'GeoPandas : {gpd.__version__}')

Pandas    : 1.1.5
GeoPandas : 0.8.0


## Set directory

In [3]:
computer_path = '/export/miro/ndeffense/LBRAT2104/'
grp_letter    = 'X'

# Directory for all work files
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'


roi_path = f'{work_path}ROI/'

Path(roi_path).mkdir(parents=True, exist_ok=True)


# EPSG of your ROI

crs_dst = 'EPSG:32631'    # WGS 84 / UTM zone 31N

roi_name      = 'extent_roi'
roi_extension = '.shp'

roi_file = f'{roi_path}{roi_name}_{crs_dst[5:]}{roi_extension}'

## Set coordinates of your ROI - be carefull of the crs_src you chose!


In [8]:
# EPSG of your coordinates
crs_src = 'EPSG:32631'

ulx = 627263.7
uly = 5596175.1
lrx = 637115
lry = 5590485.2

# WGS84 - EPSG:4326

ulx_4326 = 4.7946375
uly_4326 = 50.5037005
lrx_4326 = 4.9313909
lry_4326 = 50.4503348

bbox = ulx_4326, uly_4326, lrx_4326, lry_4326

In [12]:
IPython.display.GeoJSON(BBox(bbox,crs=4326).get_geojson())

<IPython.display.GeoJSON object>

## Build shapefile

In [4]:
# Create a list with the longitude coordinates (x)
lon_point_list = [ulx, lrx, lrx, ulx, ulx]


# Create a list with the latitude coordinates (y)
lat_point_list = [uly, uly, lry, lry, uly]


# Create a polygon object from the two list (lon and lat)
polygon_geom = Polygon(zip(lon_point_list, lat_point_list))


# Create a GeoDataFrame with the polygon object
polygon_src = gpd.GeoDataFrame(index=[0], crs=crs_src, geometry=[polygon_geom])       


# Reproject the GeoDataFrame to the destination CRS (needed only if you set your coordinates in WGS 84)
polygon_dst = polygon_src.to_crs(crs_dst)

# Check CRS of your polygon
print(f'ROI CRS : {polygon_dst.crs} \n')

# Write the GeoDataFrame in a shapefile if not exists

if not os.path.isfile(roi_file):
    polygon_dst.to_file(filename=roi_file, driver='ESRI Shapefile')
    print(f'A new vector file is created : {roi_file}')
else:
    print('The ROI vector file already exists --> delete it or change the variable "roi_name" if you want to create a new one')


ROI CRS : EPSG:32631 

The ROI vector file already exists --> delete it or change the variable "roi_name" if you want to create a new one
